In [ ]:
from torch import nn
from .utils import load_state_dict_from_url


__all__ = ['MobileNetV2', 'mobilenet_v2']


model_urls = {
    'mobilenet_v2': 'https://download.pytorch.org/models/mobilenet_v2-b0353104.pth',
}

In [ ]:
# channel수를 무조건 8로 나누어 떨어지게 만드는 함수
def _make_divisible(v, divisor, min_value=None):
    """
     이 기능은 원래 tf repo에서 가져온 것입니다.
     모든 레이어가 8로 나눌 수 있는 채널 번호를 갖도록 합니다.
     여기에서 볼 수 있습니다.
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # 반내림이 10% 이상 줄어들지 않도록 하십시오.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

# Convolution + 배치정규화 + ReLU
class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1, norm_layer=None):
        padding = (kernel_size - 1) // 2
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            norm_layer(out_planes),
            nn.ReLU6(inplace=True)
        )

#
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, norm_layer=None):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2] # stride는 무조건 1,2 둘중 하나다.

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        # expansion factor를 이용하여 channel을 확장합니다.
        hidden_dim = int(round(inp * expand_ratio))
        
        # stride가 1인 경우에만 residual block을 사용합니다
        # skip connection을 사용하는 경우 input과 output의 크기가 같아야 합니다
        self.use_res_connect = self.stride == 1 and inp == oup # skip connection이 가능한지 확인 True or False 
        
        # Inverted Residual 연산
        layers = []
        if expand_ratio != 1:
            # point-wise convolution(확장)
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1, norm_layer=norm_layer))
        layers.extend([
            # depth-wise convolution
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim, norm_layer=norm_layer),
            # point-wise linear convolution(축소)
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            norm_layer(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        # use_res_connect인 경우만 connection을 연결합니다.
        # use_res_connect : stride가 1이고 input과 output의 채널 수가 같은 경우 True
        if self.use_res_connect:
            return x + self.conv(x) # skip connection (element wise sum)
        else:
            return self.conv(x)
